## 常见的优化有以下几个方面：
* SQL及索引
* 数据库表结构
* 系统配置
* 硬件
<br/>
上面这些越来越成本高，但是优化效果越来越差。

### 如何发现有问题的SQL？
使用MYSQL慢查询日志对有效率问题的SQL进行监控：
* `show variables like 'slow_query_log'`
* `set global slow_query_log_file='/home/mysql/sql_log/mysql-slow.log'`这个是指定了慢查询的日志的位置在哪里。
* `set global log_queries_not_using_indexs=on;`指定是否把没有使用索引的SQL记录到慢查询日志中，通常是需要的。
* `set global long_query_time=1`指定把大于多少秒的SQL记录到慢查询日志中。

#### 慢查询日志包含的内容
* 执行SQL的主机信息:# User@Host:root[root]@localhost[]
* SQL的执行信息: # Query_time:0.00024 Lock_time:0.000000 Rows_sent:0 Rows_examined:0
* SQL执行时间: SET timestamp=1408275856;
* SQL的内容: select CONCAT();

#### 慢查日志的分析工具
* mysqldumpslow(MySQL官方的，会随着MySQL自动安装到我们的服务器中， -h来显示说明)